In [140]:
from my_utils import stardisting as sd

from skimage.io import imread, imsave
import numpy as np
import os
import matplotlib.pyplot as plt

model = sd.load_model(model_path=r"\\10.99.68.53\Digital pathology image lib\_Image libraries for training\2023-05-09 Published HE Nuclei Datasets\StarDist Segmentation Models\SD_HE_20x")
img_folder = r"\\10.99.68.53\Digital pathology image lib\_Image libraries for training\2023-05-09 Published HE Nuclei Datasets\20x Native\JHU\images"
msk_folder = r"\\10.99.68.53\Digital pathology image lib\_Image libraries for training\2023-05-09 Published HE Nuclei Datasets\20x Native\JHU\masks"
tile_name = 'FallopianTube_train_1.tif'
img = imread(os.path.join(img_folder, tile_name))

base_model.py (198): output path for model already exists, files may be overwritten: \\10.99.68.53\Digital pathology image lib\_Image libraries for training\2023-05-09 Published HE Nuclei Datasets\StarDist Segmentation Models\SD_HE_20x


Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.694669, nms_thresh=0.3.
Overriding defaults: Thresholds(prob=0.694668575772041, nms=0.3) 



In [141]:
overlap = 128
img_top = img[0: 256 + overlap // 2, :]
img_bot = img[256 - overlap // 2:, :]
msk_top, details_top = model.predict_instances(img_top / 255)
msk_bot, details_bot = model.predict_instances(img_bot / 255)

In [142]:
def erase_bottom_edge_objects(mask, results, ovrlp):
    blackouts = []
    for i, centroid in enumerate(results['points']):
        x, y = centroid[0], centroid[1]
        if x >= len(mask) - ovrlp // 2:
            blackouts.append(i)
    # Black out edge objects on the label.
    label_blackouts = np.asarray(blackouts) + 1
    mask[np.isin(mask, label_blackouts)] = 0
    # Remap the integer values to close the counting gaps created by blackouts
    uniques = np.unique(mask)
    remap = {unique: i for i, unique in enumerate(uniques)}
    mask = np.vectorize(remap.get)(mask)
    return mask

def erase_top_edge_objects(mask, results, ovrlp):
    blackouts = []
    for i, centroid in enumerate(results['points']):
        x, y = centroid[0], centroid[1]
        if x < ovrlp // 2:
            blackouts.append(i)
    # Black out edge objects on the label.
    label_blackouts = np.asarray(blackouts) + 1
    mask[np.isin(mask, label_blackouts)] = 0
    # Remap the integer values to close the counting gaps created by blackouts
    uniques = np.unique(mask)
    remap = {unique: i for i, unique in enumerate(uniques)}
    mask = np.vectorize(remap.get)(mask)
    return mask

In [143]:
imsave(os.path.join(img_folder, 'img_top.tif'), img_top)
imsave(os.path.join(img_folder, 'img_bot.tif'), img_bot)

In [144]:
imsave(os.path.join(img_folder, 'msk_top_full.tif'), msk_top.astype(np.uint8))
msk_top = erase_bottom_edge_objects(msk_top, details_top, overlap)
imsave(os.path.join(img_folder, 'msk_top_edged.tif'), msk_top.astype(np.uint8))

In [145]:
imsave(os.path.join(img_folder, 'msk_bot_full.tif'), msk_bot.astype(np.uint8))
msk_bot = erase_top_edge_objects(msk_bot, details_bot, overlap)
imsave(os.path.join(img_folder, 'msk_bot_edged.tif'), msk_bot.astype(np.uint8))

In [146]:
msk_black = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
imsave(os.path.join(img_folder, 'msk_blank.tif'), msk_black)

3677617039.py (2): \\10.99.68.53\Digital pathology image lib\_Image libraries for training\2023-05-09 Published HE Nuclei Datasets\20x Native\JHU\images\msk_blank.tif is a low contrast image


In [147]:
mask_bool = (msk_top != 0)
msk_black[0: 256 + overlap // 2, :][mask_bool] = msk_top[mask_bool]
imsave(os.path.join(img_folder, 'msk_top_only.tif'), msk_black)

mask_bool = (msk_bot != 0)
msk_black[256 - overlap // 2:, :][mask_bool] = msk_bot[mask_bool]
imsave(os.path.join(img_folder, 'msk_combine.tif'), msk_black)